# Data exploration of Climate Risk Viewer

In [1]:
#libraries

import pandas as pd
import numpy as np
import geopandas as gpd
import requests
import pyarrow

# To do

## Select a few layers we want to work with census

## identify values that we can transform

## Find features that will help us merge the data

## Calculate an index based on weight of each value that is overlaped

## Map the index 

In [2]:
# downloading files from the CRV


58500

In [2]:
#trying to identify counties by latitude/longitude coordinates using TIGER/Line Shapefile, 2019, nation, U.S., Current County and Equivalent National Shapefile from data.gov

# Load the shapefile
gdf = gpd.read_file('tl_2019_us_county/tl_2019_us_county.shp') #file is too large to commit; download from https://catalog.data.gov/dataset/tiger-line-shapefile-2019-nation-u-s-current-county-and-equivalent-national-shapefile

gdf.head()

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,31,039,00835841,31039,Cuming,Cuming County,06,H1,G4020,NaN,NaN,NaN,A,1477652222,10690952,+41.9158651,-096.7885168,"POLYGON ((-97.01952 42.00410, -97.01952 42.004..."
1,53,069,01513275,53069,Wahkiakum,Wahkiakum County,06,H1,G4020,NaN,NaN,NaN,A,680962890,61582307,+46.2946377,-123.4244583,"POLYGON ((-123.43639 46.23820, -123.44759 46.2..."
2,35,011,00933054,35011,De Baca,De Baca County,06,H1,G4020,NaN,NaN,NaN,A,6016819475,29089486,+34.3592729,-104.3686961,"POLYGON ((-104.56739 33.99757, -104.56772 33.9..."
3,31,109,00835876,31109,Lancaster,Lancaster County,06,H1,G4020,339,30700,NaN,A,2169270569,22849484,+40.7835474,-096.6886584,"POLYGON ((-96.91075 40.78494, -96.91075 40.790..."
4,31,129,00835886,31129,Nuckolls,Nuckolls County,06,H1,G4020,NaN,NaN,NaN,A,1489645188,1718484,+40.1764918,-098.0468422,"POLYGON ((-98.27367 40.08940, -98.27367 40.089..."


In [3]:
gdf.shape

(3233, 18)

In [4]:
gdf_CA = gdf[gdf['STATEFP']=='06']

In [5]:
gdf_CA.shape

(58, 18)

In [6]:
gdf_CA.to_csv('gdf_CA.csv')
#to write to shapefile format
#gdf_CA.to_file("California.shp")

In [7]:
gdf_CA.to_parquet('gdf_CA.parquet.gzip',
              compression='gzip')  
#pd.read_parquet('df.parquet.gzip')  

In [15]:
def get_county_name(latitude, longitude):
    point = gpd.GeoDataFrame(geometry=gpd.points_from_xy([longitude], [latitude]))
    county = gdf_CA[gdf_CA.contains(point.unary_union)]
    return county['NAME'].iloc[0] if not county.empty else None

# Example usage
latitude = 37.7749  # Example latitude
longitude = -122.4194  # Example longitude

county_name = get_county_name(latitude, longitude)
print("County Name:", county_name)


County Name: San Francisco


Exploring NorWeST_Stream_Temperatures_2040s_(Feature_Layer) downloaded as csv file from https://data-usfs.hub.arcgis.com/datasets/0cd39b50f7bb4539aac2a472bc93afe0_1/explore?location=39.832923%2C-112.988051%2C5.95

In [6]:
stream = pd.read_csv("NorWeST_Stream_Temperatures_2040s_Feature_Layer.csv")

In [7]:
stream.head()

,OBJECTID,OBSPRED_ID,S1_93_11,S30_2040D,S32_2080D,COMID,GNIS_NAME,NHDV,SHAPELEN
0,49243889,100189,21.30,21.65,21.91,15934665,,2,0.010215
1,49243890,100190,21.41,21.76,22.03,15934665,,2,0.010162
2,49243891,100191,21.50,21.85,22.11,15934665,,2,0.010135
3,49243892,100192,21.58,21.93,22.19,15934665,,2,0.009902
4,49243893,100193,21.62,21.98,22.24,15934665,,2,0.009975
